##### `Ridge & Lasso`
1. Ridge penalty is applied on square of coefficient
2. Lasso penalty is applied on absolute values of coefficient

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

##### `Problem Stmt: Estimate Weight(Col) based on other Cols in File`

In [1]:
#Step-1: Data Ingestion
import pandas as pd
df = pd.read_csv("Cars93.csv", na_values = ["", "NA"], keep_default_na = False)
df.head(1)

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra


In [5]:
#Step-2: Data Sanity - Duplicates Removal
# Duplicate Data
duplicate_count = df.duplicated().sum()

if duplicate_count > 0:
    print(f'Duplicates Found: {duplicate_count}')
    print('Removing Duplicates...')
    df = df.drop_duplicates(keep = "first").reset_index(drop = True)
    print('Removed Duplicates...')
    df.shape
else:
    print('No Duplicates Found')



No Duplicates Found


In [6]:
#Step-3: Separate X and Y
X = df.drop(columns = ["id", "Weight"])
Y = df["Weight"]
X.head(1)

#Step-4: Remove High Unique Cat Cols
card = df.select_dtypes(include = "object").nunique() / len(df)
high_card = card[card >= 0.9]
X = X.drop(columns = high_card.index)

In [10]:
#Step-5: Train Test Split
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

#Step-6: Apply Preprocessing on X
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

num_pipe = make_pipeline(
    SimpleImputer(strategy = "median"),
    StandardScaler()
)

cat_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    StandardScaler()
)
num_cols = X.select_dtypes(include = "number").columns
cat_cols = X.select_dtypes(include = "object").columns

pre = ColumnTransformer(
    [
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols)
    ]
).set_output(transform = "pandas")

pre.fit(xtrain)

xtrain_pre = pre.transform(xtrain)
xtest_pre = pre.transform(xtest)

In [12]:
#Step-7: Build Model
from sklearn.linear_model import Ridge
rr = Ridge(alpha=1)
rr.fit(xtrain_pre, ytrain)

,"alpha alpha: {float, ndarray of shape (n_targets,)}, default=1.0Constant that multiplies the L2 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Ridge` object is not advised.Instead, you should use the :class:`LinearRegression` object.If an array is passed, penalties are assumed to be specific to thetargets. Hence they must correspond in number.",1
,"fit_intercept fit_intercept: bool, default=TrueWhether to fit the intercept for this model. If setto false, no intercept will be used in calculations(i.e. ``X`` and ``y`` are expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=NoneMaximum number of iterations for conjugate gradient solver.For 'sparse_cg' and 'lsqr' solvers, the default value is determinedby scipy.sparse.linalg. For 'sag' solver, the default value is 1000.For 'lbfgs' solver, the default value is 15000.",None
,"tol tol: float, default=1e-4The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for each solver:- 'svd': `tol` has no impact.- 'cholesky': `tol` has no impact.- 'sparse_cg': norm of residuals smaller than `tol`.- 'lsqr': `tol` is set as atol and btol of scipy.sparse.linalg.lsqr, which control the norm of the residual vector in terms of the norms of matrix and coefficients.- 'sag' and 'saga': relative change of coef smaller than `tol`.- 'lbfgs': maximum of the absolute (projected) gradient=max|residuals| smaller than `tol`... versionchanged:: 1.2 Default value changed from 1e-3 to 1e-4 for consistency with other linear models.",0.0001
,"solver solver: {'auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'}, default='auto'Solver to use in the computational routines:- 'auto' chooses the solver automatically based on the type of data.- 'svd' uses a Singular Value Decomposition of X to compute the Ridge coefficients. It is the most stable solver, in particular more stable for singular matrices than 'cholesky' at the cost of being slower.- 'cholesky' uses the standard :func:`scipy.linalg.solve` function to obtain a closed-form solution.- 'sparse_cg' uses the conjugate gradient solver as found in :func:`scipy.sparse.linalg.cg`. As an iterative algorithm, this solver is more appropriate than 'cholesky' for large-scale data (possibility to set `tol` and `max_iter`).- 'lsqr' uses the dedicated regularized least-squares routine :func:`scipy.sparse.linalg.lsqr`. It is the fastest and uses an iterative procedure.- 'sag' uses a Stochastic Average Gradient descent, and 'saga' uses its improved, unbiased version named SAGA. Both methods also use an iterative procedure, and are often faster than other solvers when both n_samples and n_features are large. Note that 'sag' and 'saga' fast convergence is only guaranteed on features with approximately the same scale. You can preprocess the data with a scaler from :mod:`sklearn.preprocessing`.- 'lbfgs' uses L-BFGS-B algorithm implemented in :func:`scipy.optimize.minimize`. It can be used only when `positive` is True.All solvers except 'svd' support both dense and sparse data. However, only'lsqr', 'sag', 'sparse_cg', and 'lbfgs' support sparse input when`fit_intercept` is True... versionadded:: 0.17 Stochastic Average Gradient descent solver... versionadded:: 0.19 SAGA solver.",'auto'
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive.Only 'lbfgs' solver is supported in this case.",False
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag' or 'saga' to shuffle the data.See :term:`Glossary ` for details... versionadded:: 0.17 `random_state` to support Stochastic Average Gradient.",None


In [14]:
rr.score(xtrain_pre, ytrain), rr.score(xtest_pre, ytest)

(0.9700326911563658, 0.9323316887220762)

In [34]:
#Step-8: Hyper paramter tuning
from sklearn.model_selection import GridSearchCV
alphas = {"alpha": [0.1, 0.2, 1, 10, 50, 100, 200, 500]}
rr = Ridge()
rscv_ridge = GridSearchCV(rr, alphas, scoring="r2", cv=5)
rscv_ridge.fit(xtrain_pre, ytrain)
print(f'Ridge Best parameters: {rscv_ridge.best_params_}')
print(f'Ridge Best Score: {rscv_ridge.best_score_}')
best_ridge = rscv_ridge.best_estimator_
print(f'Ridge Best Estimator: {best_ridge}')
ytrain_pred = best_ridge.predict(xtrain_pre)
print(f'ytrain pred with Best Ridge: {ytrain_pred[0:2]}')
ytest_pred = best_ridge.predict(xtest_pre)
print(f'ytest pred with Best Ridge: {ytest_pred[0:2]}')

Ridge Best parameters: {'alpha': 10}
Ridge Best Score: 0.928904369376218
Ridge Best Estimator: Ridge(alpha=10)
ytrain pred with Best Ridge: [3810.53246426 3699.62408947]
ytest pred with Best Ridge: [2942.38293133 2317.9397172 ]


In [27]:
#Step-9: Evaluate Model
from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

def evaluate_metrics(model, x, y):
    ypred = model.predict(x)
    rmse = root_mean_squared_error(y, ypred)
    mae = mean_absolute_error(y, ypred)
    mape = mean_absolute_percentage_error(y, ypred)
    r2 = r2_score(y, ypred)
    print(f"RMSE : {rmse:.2f}")
    print(f"MAE : {mae:.2f}")
    print(f"MAPE : {mape:.2%}")
    print(f"R2 : {r2:.2%}")

print("Train Results : ")
evaluate_metrics(best_ridge, xtrain_pre, ytrain)
print("\nTest Results : ")
evaluate_metrics(best_ridge, xtest_pre, ytest)



Train Results : 
RMSE : 112.67
MAE : 88.04
MAPE : 2.89%
R2 : 96.33%

Test Results : 
RMSE : 148.59
MAE : 113.32
MAPE : 3.96%
R2 : 93.31%


In [28]:
#Step-10: Model Inference (Out of Sample Prediction)
xnew = pd.read_csv("sample.csv", na_values = ["", "NA"], keep_default_na = False)
xnew = xnew.drop(columns = high_card.index)
xnew_pre = pre.transform(xnew)
preds = best_ridge.predict(xnew_pre)
print('\nOut of Sample: Predicted Weights...')
print(preds[0:2])


Out of Sample: Predicted Weights...
[3281.88136086 2679.53254481]


##### `Lasso - L1 (Penalty applied on absolute value of coefficients)`

In [29]:
from sklearn.linear_model import Lasso

model = Lasso(alpha= 0.1)
model.fit(xtrain_pre, ytrain)

,"alpha alpha: float, default=1.0Constant that multiplies the L1 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Lasso` object is not advised.Instead, you should use the :class:`LinearRegression` object.",0.1
,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"precompute precompute: bool or array-like of shape (n_features, n_features), default=FalseWhether to use a precomputed Gram matrix to speed upcalculations. The Gram matrix can also be passed as argument.For sparse input this option is always ``False`` to preserve sparsity.",False
,"copy_X copy_X: bool, default=TrueIf ``True``, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=1000The maximum number of iterations.",1000
,"tol tol: float, default=1e-4The tolerance for the optimization: if the updates are smaller or equal to``tol``, the optimization code checks the dual gap for optimality and continuesuntil it is smaller or equal to ``tol``, see Notes below.",0.0001
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fit asinitialization, otherwise, just erase the previous solution.See :term:`the Glossary `.",False
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive.",False
,"random_state random_state: int, RandomState instance, default=NoneThe seed of the pseudo random number generator that selects a randomfeature to update. Used when ``selection`` == 'random'.Pass an int for reproducible output across multiple function calls.See :term:`Glossary `.",None
,"selection selection: {'cyclic', 'random'}, default='cyclic'If set to 'random', a random coefficient is updated every iterationrather than looping over features sequentially by default. This(setting to 'random') often leads to significantly faster convergenceespecially when tol is higher than 1e-4.",'cyclic'


In [31]:
print(f"Lasso: Train Model Score: {model.score(xtrain_pre, ytrain)}")
print(f"Lasso: Test Model Score: {model.score(xtest_pre, ytest)}")



Lasso: Train Model Score: 0.9705443640720169
Lasso: Test Model Score: 0.9314836046882216


In [32]:
alphas = {
    "alpha": [0.1, 1, 10, 100, 200, 500, 1000]
}
gscv_lasso = GridSearchCV(
    model, alphas, cv=5, scoring="r2"
)
gscv_lasso.fit(xtrain_pre, ytrain)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Lasso(alpha=0.1)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'alpha': [0.1, 1, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'r2'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate parameter indexes are 

In [37]:
print(f'Lasso Best Parameters: {gscv_lasso.best_params_}')
print(f'Lasso Best Score: {gscv_lasso.best_score_}')
best_lasso = gscv_lasso.best_estimator_
print(f'Lasso Best Estimator: {best_lasso}')

Lasso Best Parameters: {'alpha': 1}
Lasso Best Score: 0.929679262747895
Lasso Best Estimator: Lasso(alpha=1)


In [41]:
print(f'Lasso Train Score: {best_lasso.score(xtrain_pre, ytrain)}')
print(f'Lasso Test Score: {best_lasso.score(xtest_pre, ytest)}')

Lasso Train Score: 0.9701324385598128
Lasso Test Score: 0.934237913683939


In [43]:
# Evaluate Lasso for Train & Test
print("Lasso Train Results...")
evaluate_metrics(best_lasso, xtrain_pre, ytrain)
print("\nLasso Test Results...")
evaluate_metrics(best_lasso, xtest_pre, ytest)


Lasso Train Results...
RMSE : 101.63
MAE : 80.67
MAPE : 2.66%
R2 : 97.01%

Lasso Test Results...
RMSE : 147.36
MAE : 112.70
MAPE : 3.97%
R2 : 93.42%


In [45]:
#Model Inference (Out of sample) for Lasso
xnew = pd.read_csv("sample.csv", na_values = ["", "NA"], keep_default_na=False)
preds = best_lasso.predict(xnew_pre)
preds[0:5].round(2)

array([3300.01, 2689.39, 3093.31, 2982.97, 2243.19])

In [46]:
# Save & Load Model